<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Catherine Benavides
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Cornejo


### **Link de repositorio de GitHub:** `https://github.com/diegonachon/mds7202/tree/main/lab4`

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

**<span style="color:red">DISCLAIMER </span>**

**<span style="color:red">POR TEMAS DE ALMACENAMIENTO, SE COMENTARON LOS GRÁFICOS GENERADOS</span>**

**<span style="color:red">(EL NOTEBOOK PESABA MÁS DE 100 MB CON ELLOS Y NO SE PODÍA SUBIR A GITHUB)</span>**

In [1]:
# !pip install pandas --upgrade

In [2]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go
# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [4]:
df_retail = pd.read_pickle('online_retail_data.pickle')
df_retail.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,ID489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,ID489434,79323P,PINK CHERRY LIGHTS,12.0,NaT,6.75,13085.0,United Kingdom
2,ID489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,ID489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,ID489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [5]:
import plotly.express as px

def explore_data(dataframe_in):
    # Price histogram
    fig1 = px.histogram(dataframe_in, x = "Price", title = 'Price Histogram')
    fig1.show()

    # Quantity histogram
    fig2 = px.histogram(dataframe_in, x='Quantity', title='Quantity Histogram')
    fig2.show()
    
    # Nulls per column
    null_counts = dataframe_in.isnull().sum()
    print("Null values per column")
    print(null_counts)

In [1]:
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).

**Respuesta:**

In [7]:
class IQR(BaseEstimator, TransformerMixin):
    def __init__(self, lambda_value = 1):
        self.lambda_value = lambda_value
        self.numeric_features = None
        self.lower_bound = None
        self.upper_bound = None
    
    def fit(self, X, y=None):
        self.numeric_features = X.select_dtypes(include=['int', 'float']).columns
        Q1 = X[self.numeric_features].quantile(0.25)
        Q3 = X[self.numeric_features].quantile(0.75)
        IQR = Q3 - Q1
        self.lower_bound = Q1 - self.lambda_value * IQR
        self.upper_bound = Q3 + self.lambda_value * IQR
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        
        
        for col in self.numeric_features:
            X_copy[(X_copy[col] <= self.lower_bound[col]) | (X_copy[col] >= self.upper_bound[col])] = np.nan
            
        return X_copy

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Ver hint al final. [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador 'passthrough' está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación.

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [3]:
numerical_columns = ['Price', 'Quantity']
categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate','Customer ID', 'Country']

numeric_transformations = Pipeline([('iqr', IQR(lambda_value = 3))])

column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)
                                        ])

df_iqr = pd.DataFrame(column_transformer.fit_transform(df_retail), 
             columns = ['Price', 'Quantity', 'Invoice', 'StockCode', 
                        'Description', 'InvoiceDate','Customer ID', 'Country'])

# explore_data(df_retail)
# explore_data(df_iqr)

A partir del valor de lambda, se van eliminando aquellos valores atípicos de las variables, tanto en *Price* como en *Quantity*, haciendo posible observar de mejor manera la distribución que tienen. Así, mientras más alto sea lambda, se tiene que se consideran como posibles más valores atípicos, por lo que se consideran más datos en comparación a un lambda pequeño, que será más estricto en cuanto a los outliers existentes.

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la moda [link](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media [link](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Aplicar el ColumnTransformer actualizado a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por `knn_imputer` [link](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. [1 punto]
6. Comprar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comprar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

**Respuesta:**

In [9]:
from sklearn.impute import SimpleImputer

**P1**

In [10]:
categoric_transformations = Pipeline([('mode_imputer', SimpleImputer(strategy='most_frequent'))])


**P2**

In [11]:
numeric_transformations.steps.append(('mean_imputer', SimpleImputer(strategy='mean')))


**P3**

In [12]:
column_transformer = ColumnTransformer([('numeric', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)
                                       ])

df_mean_imputer = pd.DataFrame(column_transformer.fit_transform(df_retail), 
                               columns = ['Price', 'Quantity', 'Invoice', 'StockCode', 
                                          'Description', 'InvoiceDate','Customer ID', 'Country'])

**P4**

In [4]:
explore_data(df_iqr)
explore_data(df_mean_imputer)

Las principal diferencia en ambas variables se observa en el aumento de un valor en específico (una barra) en los gráficos con la imputación de la media de las variables. En el caso de *Price*, se tiene que aumentan los valores entre 2,73 y 2,79; mientras que en *Quantity* es 7,49. Por otro lado, se tiene que ya no hay valores nulos en ninguna columna.

**P5**

In [14]:
from sklearn.impute import KNNImputer

numeric_transformations.steps[-1] = ('knn_imputer', KNNImputer())

df_knn_imputer = pd.DataFrame(column_transformer.fit_transform(df_retail), 
                              columns = ['Price', 'Quantity', 'Invoice', 'StockCode', 
                                         'Description', 'InvoiceDate','Customer ID', 'Country'])

**P6**

In [5]:
explore_data(df_iqr)
explore_data(df_knn_imputer)

Se observa que en el caso del knn imputer, existen más barras que en el mean, tanto en la variable *Price*, como en la variable *Quantity*. Esto se debe a que la imputación se basa en los vecinos más cercanos (y no solamente con la media, como se hace en el caso anterior).

**P7**

El método de imputación que es mejor corresponde al KNN, ya que el algoritmo considera las demás variables para la estimación, por lo que corresponderían a valores *''más realistas''*.

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(3 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [16]:
def custom_features(dataframe_in):
    
    # Customer ID
    customers = list(df_retail['Customer ID'].unique().sort_values())
    
    # Lenght
    length = dataframe_in.groupby('Customer ID')['InvoiceDate']\
                            .apply(lambda x: (x.max() - x.min()).days).sort_index()
    
    # Recency
    recency = dataframe_in.groupby('Customer ID')['InvoiceDate'].max()\
                            .apply(lambda x: (datetime.datetime.now() - x).days).sort_index()
    
    # Monetary
    dataframe_in['Quantity_Price'] = dataframe_in['Quantity'] * dataframe_in['Price']
    total_purchases = dataframe_in.groupby('Customer ID')['Quantity_Price'].sum()
    total_sessions  = dataframe_in.groupby('Customer ID')['Invoice'].nunique()
    monetary = (total_purchases/total_sessions).sort_index()
    
    # Frecuency
    frecuency = total_sessions.copy().sort_index()
    
    # Periodicity
    df_unique = dataframe_in.sort_values(by=['Customer ID', 'InvoiceDate'])\
                                .drop_duplicates(subset = ['Customer ID', 'Invoice'])
    df_unique['IVT'] = df_unique.groupby('Customer ID')['InvoiceDate'].diff()
    df_unique['Customer ID'] = df_unique['Customer ID'].shift(-1)
    df_unique['Invoice'] = df_unique['Invoice'].shift(-1)
    periodicity = df_unique.groupby('Customer ID')['IVT'].std().sort_index()
    periodicity = periodicity.apply(lambda x: x.days).fillna(0)
    
    
    df_lrmfp = pd.DataFrame()
    df_lrmfp['Customer ID'] = customers
    df_lrmfp['Length'] = length.values
    df_lrmfp['Recency'] = recency.values
    df_lrmfp['Monetary'] = monetary.values
    df_lrmfp['Frecuency'] = frecuency.values
    df_lrmfp['Periodicity'] = periodicity.values
    df_lrmfp = df_lrmfp.reset_index(drop = True)
    
    return df_lrmfp

In [17]:
custom_features(df_retail)

,Customer ID,Length,Recency,Monetary,Frecuency,Periodicity
0,12346.0,196.0,5052.0,33.896364,11,12.0
1,12347.0,37.0,4890.0,628.935000,2,0.0
2,12348.0,0.0,4961.0,222.160000,1,0.0
3,12349.0,181.0,4930.0,882.130000,3,0.0
4,12351.0,0.0,4898.0,300.930000,1,0.0
...,...,...,...,...,...,...
4309,18283.0,275.0,4905.0,101.836667,6,62.0
4310,18284.0,0.0,4954.0,431.080000,1,0.0
4311,18285.0,0.0,5183.0,427.000000,1,0.0
4312,18286.0,247.0,4999.0,585.675000,2,0.0


#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

In [18]:
# Como el df queda en array, se crea una funcion que permita transformar esta
# nuevamente en df despues del col_transformer.

def to_dataframe(X):
    X = pd.DataFrame(X, columns=['Price', 'Quantity', 'Invoice', 
                                 'StockCode', 'Description', 'InvoiceDate', 
                                 'Customer ID', 'Country'])
    return X

In [19]:
retail_pipeline = Pipeline([
    ('col_transformer', column_transformer),
    ('to_dataframe', FunctionTransformer(to_dataframe)),
    ('custom_features', FunctionTransformer(custom_features))
])

In [20]:
df_custom = retail_pipeline.fit_transform(df_retail)

In [21]:
df_custom

,Customer ID,Length,Recency,Monetary,Frecuency,Periodicity
0,12346.0,196,5052,33.896364,11,12.0
1,12347.0,274,4890,664.752,2,0.0
2,12348.0,0,4961,222.16,1,0.0
3,12349.0,234,4930,692.288037,3,0.0
4,12351.0,0,4898,308.704016,1,0.0
...,...,...,...,...,...,...
4309,18283.0,275,4905,103.728333,6,113.0
4310,18284.0,0,4954,398.352063,1,0.0
4311,18285.0,0,5183,204.672047,1,0.0
4312,18286.0,247,4999,605.925031,2,0.0


**DISCLAIMER**

Piden usar explore_data para ver las nuevas variables, sin embargo pidieron que esta función mostrara
Quantity y Price. Asique, en vez de usar explore_data, se uso plotly para mostrar estas variables y
comentarlas.

**FIN DISCLAIMER**

In [22]:
df_custom.head()

,Customer ID,Length,Recency,Monetary,Frecuency,Periodicity
0,12346.0,196,5052,33.896364,11,12.0
1,12347.0,274,4890,664.752,2,0.0
2,12348.0,0,4961,222.16,1,0.0
3,12349.0,234,4930,692.288037,3,0.0
4,12351.0,0,4898,308.704016,1,0.0


In [6]:
length = px.histogram(df_custom, x = "Length", title = 'Length Histogram')
length.show()

In [7]:
recency = px.histogram(df_custom, x = "Recency", title = 'Recency Histogram')
recency.show()

In [8]:
monetary = px.histogram(df_custom, x = "Monetary", title = 'Monetary Histogram')
monetary.show()

In [9]:
frecuency = px.histogram(df_custom, x = "Frecuency", title = 'Frecuency Histogram')
frecuency.show()

In [10]:
periodicity = px.histogram(df_custom, x = "Periodicity", title = 'Periodicity Histogram')
periodicity.show()

**Respuesta**

- **Length**: Entre 0 y 9 días se concentra la mayor cantidad de clientes, es decir, hay muchos clientes nuevos. Por otro lado, se observan dos tipos de cluster de clientes fieles, uno entre el rango de 210-279 días de antiguedad; y otro entre los 340-369 días de antiguedad.

- **Recency**: Hasta el momento de la toma de los datos (2010-12-09 o hace 4888 días atrás), se observa una gran concentración de clientes en los días previos, pudiéndose interpretar como que una gran cantidad de estos habían comprado recientemente. Así mismo, se observan dos especies de grupos relevantes que concentran clientes en la lejanía, siendo entre los 5140-5149 días, y entre los 5240-5269 días; lo que si se considera el 2010-12-09 como el día de referencia, serían entre los 252-261 días y 352-381 días.

- **Monetary**: Se observa que, en promedio, los clientes gastan enttre 170-200 en cada compra. Además, se tiene que presenta una distribución normal asímetrica positiva, lo que quiere decir que exiten valores muy altos que tiran la media hacia la derecha.

- **Frecuency**: Se tiene que la mayor cantidad de clientes a comprado sólo una vez, por lo que correspondería a un público poco fiel. Además, se presentan algunos casos extremos, donde la frecuencia alcanza valores por sobre 100, incluso 200, siendo necesaría la revisión de estos.

- **Periodicity**: Existe una gran cantidad de clientes en el primer tramo, ya que muchos son 0 o se encuentran en valores inferiores a 4. Los correspondientes a 0, se debe a que al calcular la desviación estandar, en muchos casos hay solo 1 valor, lo que se considera 0.




### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [28]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.min_values = None
        self.max_values = None
    
    def fit(self, X, y=None):
        self.min_values = X.min()
        self.max_values = X.max()
        return self
    
    def transform(self, X, y=None):
        return (X - self.min_values) / (self.max_values - self.min_values)

#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

**Respuesta:**

In [29]:
numeric_transformations.steps.append(('min_max', MinMax()))

In [30]:
column_transformer = ColumnTransformer([('numeric', numeric_transformations, numerical_columns),
                                        ('categorical', categoric_transformations, categorical_columns)
                                       ])

In [31]:
df_minmax = pd.DataFrame(column_transformer.fit_transform(df_retail),
                         columns=['Price', 'Quantity', 'Invoice', 
                                  'StockCode', 'Description', 'InvoiceDate', 
                                  'Customer ID', 'Country'])

In [32]:
df_minmax

,Price,Quantity,Invoice,StockCode,Description,InvoiceDate,Customer ID,Country
0,0.169512,0.292683,ID489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,2009-12-01 07:45:00,13085.0,United Kingdom
1,0.164634,0.292683,ID489434,79323P,PINK CHERRY LIGHTS,2010-03-07 15:34:00,13085.0,United Kingdom
2,0.164634,0.292683,ID489434,79323W,WHITE CHERRY LIGHTS,2009-12-01 07:45:00,13085.0,United Kingdom
3,0.066803,0.182767,ID489434,22041,"RECORD FRAME 7"" SINGLE SIZE",2009-12-01 07:45:00,13085.0,United Kingdom
4,0.030488,0.585366,ID489434,21232,STRAWBERRY CERAMIC TRINKET BOX,2009-12-01 07:45:00,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
400942,0.071951,0.04878,ID538171,22271,FELTCRAFT DOLL ROSIE,2010-12-09 20:01:00,17530.0,United Kingdom
400943,0.091463,0.02439,ID538171,22750,FELTCRAFT PRINCESS LOLA DOLL,2010-12-09 20:01:00,17530.0,United Kingdom
400944,0.091463,0.02439,ID538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,2010-12-09 20:01:00,17530.0,United Kingdom
400945,0.091463,0.063415,ID538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2010-12-09 20:01:00,17530.0,United Kingdom


In [11]:
explore_data(df_retail)
explore_data(df_minmax)

**Respuesta**

Se observa de mejor manera la distribución de las variables en df_minmax que en df_retail, ya que se normalizaron según el maximo y el mínimo de estas. En el caso del Price, se presenta de mejor manera los rangos de valores menores, en comparación al retail, al igual que en Quantity.

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

La creación de pipelines ayuda en simplificar el proceso de aprendizaje automático, permitnendo combinar distintos pasos del proceso de este en una única entidad; procesamiento de datos consistentes, asegurando que los mismos pasos de preprocesamiento de datos se apliquen de manera consistente tanto a los datos de entrenamiento como de testeo; ajuste de hiperparámetros, permitiendo encontrar una combinación óptima de hiperparámetros y mejorar el rendimiento; legibilidad y reutilización del codigo, organizando de forma clara y consisa el código, facilitando su lectura y mantenimiento; y mejora en el rendimiento, al mantener al mínimo la cantidad de manipulación de datos y cálculo requerido.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>